In [2]:
import tensorflow 
from keras import backend as K
from tensorflow.python.keras.applications import ResNet50,VGG16
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.layers import Input,Dense, Flatten, GlobalAveragePooling2D,Lambda, Dropout
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import time
import os
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import random
import matplotlib.pylab as plt
%matplotlib inline

Using TensorFlow backend.


In [77]:
# Model architecture using VGG16 
left_input = Input(shape=(224, 224, 3))
right_input=Input(shape=(224,224,3))
model = Sequential()
model.add(VGG16(pooling='avg', include_top=False,weights='imagenet'))
for i in range(11):
    model.layers[0].layers[i].trainable = False

In [78]:
model.layers[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [79]:
encoded_l = model(left_input)
encoded_r = model(right_input)
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
#prediction = Dense(2,activation='sigmoid')(L1_distance)
prediction = Dense(512,activation='relu')(L1_distance)
prediction = Dropout(0.5)(prediction)
prediction = Dense(1,activation='sigmoid')(prediction)
# Indicate whether the first layer should be trained/changed or not.
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
#siamese_net.compile(loss="categorical_crossentropy",optimizer='sgd',metrics=['accuracy'])
siamese_net.compile(loss="binary_crossentropy",optimizer='sgd',metrics=['accuracy'])

In [80]:
siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 512)          14714688    input_10[0][0]                   
                                                                 input_11[0][0]                   
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 512)          0           sequential_3[1][0]               
          

In [7]:
siamese_pairs_1=[]
siamese_pairs_2=[]
non_siamese_pairs_1=[]
non_siamese_pairs_2=[]

In [8]:
def get_siamese_pairs_female_train(path1,path2):
    dirs1 = os.listdir(path1)
    dirs2=  os.listdir(path2)
    for item in dirs1:
        for item2 in dirs2:
            f1, e1 = os.path.splitext(path1+item)
            f2, e2 = os.path.splitext(path2+item2)
            if f1[27:-2] == f2[27:-2]:
                img1 = image.load_img(path1+item, target_size=(224, 224))
                img2 = image.load_img(path2+item2, target_size=(224,224))
                x1 = image.img_to_array(img1)
                x2 = image.img_to_array(img2)
                x1 = preprocess_input(x1)
                x2 = preprocess_input(x2)
                x1 = x1/255
                x2 = x2/255
                siamese_pairs_1.append(x1)
                siamese_pairs_2.append(x2)

In [9]:
# For Female Category1
get_siamese_pairs_female_train('./train/1/Female/Category1/','./train/2/Female/Category1/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))
# For Female Category2
get_siamese_pairs_female_train('./train/1/Female/Category2/','./train/2/Female/Category2/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))
# For Female Category3
get_siamese_pairs_female_train('./train/1/Female/Category3/','./train/2/Female/Category3/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))

7
7
17
17
24
24


In [10]:
def get_siamese_pairs_male_train(path1,path2):
    dirs1 = os.listdir(path1)
    dirs2=  os.listdir(path2)
    for item in dirs1:
        for item2 in dirs2:
            f1, e1 = os.path.splitext(path1+item)
            f2, e2 = os.path.splitext(path2+item2)
            if f1[25:-2] == f2[25:-2]:
                img1 = image.load_img(path1+item, target_size=(224, 224))
                img2 = image.load_img(path2+item2, target_size=(224,224))
                x1 = image.img_to_array(img1)
                x2 = image.img_to_array(img2)
                x1 = preprocess_input(x1)
                x2 = preprocess_input(x2)
                x1 = x1/255
                x2 = x2/255
                siamese_pairs_1.append(x1)
                siamese_pairs_2.append(x2)

In [11]:
# For Male Category1
get_siamese_pairs_male_train('./train/1/Male/Category1/','./train/2/Male/Category1/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))
# For Male Category2
get_siamese_pairs_male_train('./train/1/Male/Category2/','./train/2/Male/Category2/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))
# For Male Category3
get_siamese_pairs_male_train('./train/1/Male/Category3/','./train/2/Male/Category3/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))

41
41
46
46
56
56


In [12]:
def get_siamese_pairs_female_validation(path1,path2):
    dirs1 = os.listdir(path1)
    dirs2=  os.listdir(path2)
    for item in dirs1:
        for item2 in dirs2:
            f1, e1 = os.path.splitext(path1+item)
            f2, e2 = os.path.splitext(path2+item2)
            if f1[31:-2] == f2[31:-2]:
                img1 = image.load_img(path1+item, target_size=(224, 224))
                img2 = image.load_img(path2+item2, target_size=(224,224))
                x1 = image.img_to_array(img1)
                x2 = image.img_to_array(img2)
                x1 = preprocess_input(x1)
                x2 = preprocess_input(x2)
                x1 = x1/255
                x2 = x2/255
                siamese_pairs_1.append(x1)
                siamese_pairs_2.append(x2)

In [13]:
# For Female Category1
get_siamese_pairs_female_validation('./validation/1/Female/Category1/','./validation/2/Female/Category1/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))
# For Female Category2
get_siamese_pairs_female_validation('./validation/1/Female/Category2/','./validation/2/Female/Category2/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))
# For Female Category3
get_siamese_pairs_female_validation('./validation/1/Female/Category3/','./validation/2/Female/Category3/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))

63
63
70
70
80
80


In [14]:
def get_siamese_pairs_male_validation(path1,path2):
    dirs1 = os.listdir(path1)
    dirs2=  os.listdir(path2)
    for item in dirs1:
        for item2 in dirs2:
            f1, e1 = os.path.splitext(path1+item)
            f2, e2 = os.path.splitext(path2+item2)
            if f1[29:-2] == f2[29:-2]:
                img1 = image.load_img(path1+item, target_size=(224, 224))
                img2 = image.load_img(path2+item2, target_size=(224,224))
                x1 = image.img_to_array(img1)
                x2 = image.img_to_array(img2)
                x1 = preprocess_input(x1)
                x2 = preprocess_input(x2)
                x1 = x1/255
                x2 = x2/255
                siamese_pairs_1.append(x1)
                siamese_pairs_2.append(x2)

In [15]:
# For Male Category1
get_siamese_pairs_male_validation('./validation/1/Male/Category1/','./validation/2/Male/Category1/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))
# For Male Category2
get_siamese_pairs_male_validation('./validation/1/Male/Category2/','./validation/2/Male/Category2/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))
# For Male Category3
get_siamese_pairs_male_validation('./validation/1/Male/Category3/','./validation/2/Male/Category3/')
print(len(siamese_pairs_1))
print(len(siamese_pairs_2))

85
85
92
92
105
105


In [16]:
def get_non_siamese_pairs_female_train(path1,path2):
    dirs1 = os.listdir(path1)
    dirs2=  os.listdir(path2)
    for item in dirs1:
        li_1=[]
        li_2=[]
        for item2 in dirs2:
            f1, e1 = os.path.splitext(path1+item)
            f2, e2 = os.path.splitext(path2+item2)
            if f1[27:-2] != f2[27:-2]:
                img1 = image.load_img(path1+item, target_size=(224, 224))
                img2 = image.load_img(path2+item2, target_size=(224,224))
                x1 = image.img_to_array(img1)
                x2 = image.img_to_array(img2)
                x1 = preprocess_input(x1)
                x2 = preprocess_input(x2)
                x1 = x1/255
                x2 = x2/255
                li_1.append(x1)
                li_2.append(x2)
        n=random.randint(0,len(li_1)-1)
        non_siamese_pairs_1.append(li_1[n])
        non_siamese_pairs_2.append(li_2[n])

In [17]:
# For Female Category1
get_non_siamese_pairs_female_train('./train/1/Female/Category1/','./train/2/Female/Category1/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))
# For Female Category2
get_non_siamese_pairs_female_train('./train/1/Female/Category2/','./train/2/Female/Category2/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))
# For Female Category3
get_non_siamese_pairs_female_train('./train/1/Female/Category3/','./train/2/Female/Category3/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))

7
7
17
17
24
24


In [18]:
def get_non_siamese_pairs_male_train(path1,path2):
    dirs1 = os.listdir(path1)
    dirs2=  os.listdir(path2)
    for item in dirs1:
        li_1=[]
        li_2=[]
        for item2 in dirs2:
            f1, e1 = os.path.splitext(path1+item)
            f2, e2 = os.path.splitext(path2+item2)
            if f1[25:-2] != f2[25:-2]:
                img1 = image.load_img(path1+item, target_size=(224, 224))
                img2 = image.load_img(path2+item2, target_size=(224,224))
                x1 = image.img_to_array(img1)
                x2 = image.img_to_array(img2)
                x1 = preprocess_input(x1)
                x2 = preprocess_input(x2)
                x1 = x1/255
                x2 = x2/255
                li_1.append(x1)
                li_2.append(x2)
        n = random.randint(0,len(li_1)-1)
        non_siamese_pairs_1.append(li_1[n])
        non_siamese_pairs_2.append(li_2[n])

In [19]:
# For Male Category1
get_non_siamese_pairs_male_train('./train/1/Male/Category1/','./train/2/Male/Category1/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))
# For Male Category2
get_non_siamese_pairs_male_train('./train/1/Male/Category2/','./train/2/Male/Category2/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))
# For Male Category3
get_non_siamese_pairs_male_train('./train/1/Male/Category3/','./train/2/Male/Category3/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))

41
41
46
46
56
56


In [20]:
def get_non_siamese_pairs_female_validation(path1,path2):
    dirs1 = os.listdir(path1)
    dirs2=  os.listdir(path2)
    for item in dirs1:
        if(os.path.isfile(path1+item)):
            li_1=[]
            li_2=[]
            for item2 in dirs2:
                if(os.path.isfile(path2+item2)):
                    f1, e1 = os.path.splitext(path1+item)
                    f2, e2 = os.path.splitext(path2+item2)
                    if f1[31:-2] != f2[31:-2]:
                        img1 = image.load_img(path1+item, target_size=(224, 224))
                        img2 = image.load_img(path2+item2, target_size=(224,224))
                        x1 = image.img_to_array(img1)
                        x2 = image.img_to_array(img2)
                        x1 = preprocess_input(x1)
                        x2 = preprocess_input(x2)
                        x1 = x1/255
                        x2 = x2/255
                        li_1.append(x1)
                        li_2.append(x2)
            n=random.randint(0,len(li_1)-1)
            non_siamese_pairs_1.append(li_1[n])
            non_siamese_pairs_2.append(li_2[n])

In [21]:
# For Female Category1
get_non_siamese_pairs_female_validation('./validation/1/Female/Category1/','./validation/2/Female/Category1/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))
# For Female Category2
get_non_siamese_pairs_female_validation('./validation/1/Female/Category2/','./validation/2/Female/Category2/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))
# For Female Category3
get_non_siamese_pairs_female_validation('./validation/1/Female/Category3/','./validation/2/Female/Category3/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))

63
63
70
70
80
80


In [22]:
def get_non_siamese_pairs_male_validation(path1,path2):
    dirs1 = os.listdir(path1)
    dirs2=  os.listdir(path2)
    for item in dirs1:
        if(os.path.isfile(path1+item)):
            li_1=[]
            li_2=[]
            for item2 in dirs2:
                if(os.path.isfile(path2+item2)):
                    f1, e1 = os.path.splitext(path1+item)
                    f2, e2 = os.path.splitext(path2+item2)
                    if f1[29:-2] != f2[29:-2]:
                        img1 = image.load_img(path1+item, target_size=(224, 224))
                        img2 = image.load_img(path2+item2, target_size=(224,224))
                        x1 = image.img_to_array(img1)
                        x2 = image.img_to_array(img2)
                        x1 = preprocess_input(x1)
                        x2 = preprocess_input(x2)
                        x1 = x1/255
                        x2 = x2/255
                        li_1.append(x1)
                        li_2.append(x2)
            n=random.randint(0,len(li_1)-1)
            non_siamese_pairs_1.append(li_1[n])
            non_siamese_pairs_2.append(li_2[n])

In [23]:
# For Male Category1
get_non_siamese_pairs_male_validation('./validation/1/Male/Category1/','./validation/2/Male/Category1/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))
# For Male Category2
get_non_siamese_pairs_male_validation('./validation/1/Male/Category2/','./validation/2/Male/Category2/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))
# For Male Category3
get_non_siamese_pairs_male_validation('./validation/1/Male/Category3/','./validation/2/Male/Category3/')
print(len(non_siamese_pairs_1))
print(len(non_siamese_pairs_2))

85
85
92
92
105
105


In [24]:
labels=[]
for item in siamese_pairs_1:
    labels.append(1)
for item in non_siamese_pairs_1:
    labels.append(0)

In [25]:
pairs_1 = siamese_pairs_1 + non_siamese_pairs_1
pairs_2 = siamese_pairs_2 + non_siamese_pairs_2

In [26]:
#from keras.utils import np_utils
#num_classes=2
#Y = np_utils.to_categorical(labels, num_classes)
Y = labels

In [27]:
#Shuffle the dataset
pairs1,pairs2,y = shuffle(pairs_1,pairs_2,Y, random_state=2)

In [28]:
len(pairs_1)

210

In [29]:
Xtrain1 = pairs1[:150]
Xtrain2 = pairs2[:150]
ytrain = y[:150]

In [30]:
Xtest1 = pairs1[150:]
Xtest2 = pairs2[150:]
ytest = y[150:]

In [31]:
Xtrain1 = np.array([Xtrain1]).reshape(150,224,224,3)
Xtrain1.shape

(150, 224, 224, 3)

In [32]:
Xtrain2 = np.array([Xtrain2]).reshape(150,224,224,3)
Xtrain2.shape

(150, 224, 224, 3)

In [89]:
t = time.time()
hist = siamese_net.fit([Xtrain1,Xtrain2], ytrain, batch_size=32, epochs=10, verbose=2,validation_split = 0.33)
#(loss, accuracy) = siamese_net.evaluate(X_test, y_test, batch_size=10, verbose=1)
#print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print('Training time: %s' % (t - time.time()))

Train on 100 samples, validate on 50 samples
Epoch 1/10
 - 199s - loss: 0.6465 - acc: 0.7100 - val_loss: 0.7130 - val_acc: 0.4600
Epoch 2/10
 - 212s - loss: 0.6458 - acc: 0.6100 - val_loss: 0.7110 - val_acc: 0.4600
Epoch 3/10
 - 220s - loss: 0.6372 - acc: 0.6500 - val_loss: 0.7227 - val_acc: 0.4600
Epoch 4/10
 - 216s - loss: 0.6173 - acc: 0.6300 - val_loss: 0.7329 - val_acc: 0.4600
Epoch 5/10
 - 221s - loss: 0.6183 - acc: 0.6200 - val_loss: 0.7289 - val_acc: 0.4600
Epoch 6/10
 - 224s - loss: 0.5996 - acc: 0.6500 - val_loss: 0.7303 - val_acc: 0.4600
Epoch 7/10
 - 207s - loss: 0.5839 - acc: 0.7400 - val_loss: 0.7568 - val_acc: 0.4600
Epoch 8/10
 - 197s - loss: 0.5620 - acc: 0.6800 - val_loss: 0.7540 - val_acc: 0.4600
Epoch 9/10
 - 209s - loss: 0.5452 - acc: 0.7200 - val_loss: 0.7518 - val_acc: 0.4600
Epoch 10/10
 - 210s - loss: 0.5363 - acc: 0.7700 - val_loss: 0.7540 - val_acc: 0.4400
Training time: -2117.3476691246033


In [90]:
r1 = siamese_net.predict([siamese_pairs_1,siamese_pairs_2])

In [91]:
r2 = siamese_net.predict([non_siamese_pairs_1,non_siamese_pairs_2])

In [113]:
len(r1[r1>=0.385])/len(r1)

0.7333333333333333

In [112]:
len(r2[r2<0.385])/len(r2)

0.6476190476190476